In [189]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken

In [244]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [245]:
!mkdir pdfs

mkdir: pdfs: File exists


In [246]:
loader=PyPDFDirectoryLoader("pdfs")

In [247]:
data=loader.load()

In [248]:
data[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing w

chunk size 500 mean every chuk has 500 max words and chunk_overlap mean 20 word can be overlap

In [249]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [250]:
text_chunks = text_splitter.split_documents(data)

In [251]:
text_chunks

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'pdfs/transformer.pdf', 'page': 0}),
 Document(page_content='aidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network

In [252]:
text_chunks[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto', metadata={'source': 'pdfs/transformer.pdf', 'page': 0})

In [253]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [254]:
len(text_chunks)

91

In [255]:
import os
os.environ["OPENAI_API_KEY"]=""
os.environ["PINECONE_API_KEY"] = ""

In [256]:
embedding=OpenAIEmbeddings()

In [257]:
embedding.embed_query("How are you")

[-0.01617186794777671,
 -0.002785683447152409,
 -0.0048103380938062,
 -0.03635182392278896,
 -0.02380752610953903,
 0.016615801453539344,
 -0.02899520861456599,
 -0.020040432685597537,
 -0.023388959966584973,
 -0.0051306045417238875,
 0.025304217279050214,
 0.00559356356959295,
 -0.010521226880541952,
 0.0039858899549636275,
 -0.010058267387011629,
 -0.015474258019960786,
 0.04375917209398404,
 -0.00925918707663889,
 0.008003489579099233,
 -0.014725912435205194,
 0.0023639466166779084,
 0.009690436900997235,
 0.0006052559121646145,
 -0.0027079950836439485,
 -0.01697094825814945,
 0.0001507194304753277,
 0.007121963942614851,
 -0.01567719971639693,
 0.0047088672455881265,
 -0.028411751819388643,
 0.0033073055492541326,
 -0.005495264340217839,
 -0.005790162959665693,
 0.0026350632170774103,
 0.007394665886774574,
 -0.02128344650173291,
 -0.009176742681849768,
 0.0030425307895567736,
 0.021854218684183448,
 0.005143288223128174,
 0.01983749192048391,
 -0.00011227160648902412,
 -0.00576479

In [258]:
len(embedding.embed_query("How are you"))

1536

In [259]:
import os
from pinecone import Pinecone, ServerlessSpec


In [260]:
pc = Pinecone(api_key='')

In [261]:
pc

In [262]:
index_name = "firstindex"

In [263]:
index = pc.Index('firstindex')

In [264]:
#for i, t in zip(range(len(text_chunks)), text_chunks):
#   query_result = embedding.embed_query(t.page_content)
#   index.upsert(
#   vectors=[
#        {
#            "id": str(i),  # Convert i to a string
#            "values": query_result, 
#            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
#        }
#    ],
#    namespace="real" 
#)
#index.describe_index_stats() 

In [265]:
query= "Attention Is All You Need"
#PineconeVectorStore.from_texts()

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)
docsearch.as_retriever()
#docsearch = PineconeVectorStore.from_existing_index(index, embedding)
answer = docsearch.similarity_search(query)
answer


[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto'),
 Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUni

In [266]:
llm=OpenAI()




In [267]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [268]:
query= "what is Attention Is All You Need"

In [269]:
results=qa.invoke(query)
results

{'query': 'what is Attention Is All You Need', 'result': ' I do not know.'}

In [270]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa.invoke({'query': user_input})
  print(f"Answer: {result['result']}")

Answer:  I don't know.
Answer:  Attention Is All You Need is a type of self-attention mechanism used in natural language processing tasks, as described in section 3.2 and used in tasks such as reading comprehension, abstractive summarization, textual entailment, and sentence representation. It is also known as intra-attention and involves relating different positions of a single sequence to compute a representation of the sequence. 
Exiting


SystemExit: 

/opt/anaconda3/envs/pinecone-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
